In [ ]:
# show the blurred image
cv2.imshow('shapes', img_normal_blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#simple thresholding
ret, thrash = cv2.threshold(img_shade_blurred2, 78 , 255, cv2.THRESH_BINARY)

In [ ]:
#cv2.THRESH_BINARY_INV 
#Siyah arkaplan beyaz obje

#cv2.THRESH_BINARY
#beyaz arkaplan siyah obje

In [ ]:
#adaptive thresholding
# block size 15: default 21 
# block size düşünce daha iyi sonuç elde ettim
daha keskin çizgiler
thresh = cv2.adaptiveThreshold(img_shade_blurred2, 255,
	cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 15, 10)
ADAPTIVE_THRESH_MEAN_C 
Python: cv.ADAPTIVE_THRESH_MEAN_C
the threshold value T(x,y) is a mean of the blockSize×blockSize neighborhood of (x,y) minus C
daha yumuşak çizgiler
ADAPTIVE_THRESH_GAUSSIAN_C 
Python: cv.ADAPTIVE_THRESH_GAUSSIAN_C
the threshold value T(x,y) is a weighted sum (cross-correlation with a Gaussian window) of 
the blockSize×blockSize neighborhood of (x,y) minus C .
The default sigma (standard deviation) is used for the
specified blockSize . See getGaussianKernel

In [ ]:
#otsu 
# gölgelerde otsu işe yaramıyor
# black background with white objects
ret,th = cv2.threshold(img_shade_blurred2,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY_INV)

In [ ]:
# first [144  49] second  [143  50]
def ssd(first, second):
    sum = 0 
    x_differ = abs(first[0]- second[0])
    y_differ = abs(first[1]- second[1])
    sum =  x_differ**2 + y_differ**2
    return sum

In [ ]:
def simplyfy_list(corners):#[[[]],[[]]] ilk bulduğumu eledim diye köşeler kayıyor
    co = len(corners) #12
    ret_list = []
    for i in range(co): #0
        flag = True
        for j in range(i+1,co):
            ssd1 = ssd(corners[i][0],corners[j][0])
            if ssd1>100:
                flag = False
        if not flag:
            ret_list.append(corners[i][0])
        
    return ret_list

In [ ]:
#p1 = [ 81  50] ...
# vector = destination-source, p1 == source,p2 == destination 
def vector_of_points(p1,p2):
    x = p2[0]-p1[0]
    y = p2[1]-p1[1]
    return [x,y]

In [ ]:
'''
yanliş açi değeri veriyor
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

p1 = [144,49]
p2 = [143,50]
p3 = [81,50]
v1 = vector_of_points(p1,p2)
v2 = vector_of_points(p1,p3)
ang = angle_between(v1,v2)
print(ang)
'''

In [ ]:
"""
import math
def angle_of_vectors(a,b,c,d):
    
     dotProduct = a*c + b*d
         # for three dimensional simply add dotProduct = a*c + b*d  + e*f 
     modOfVector1 = math.sqrt( a*a + b*b)*math.sqrt(c*c + d*d) 
         # for three dimensional simply add modOfVector = math.sqrt( a*a + b*b + e*e)*math.sqrt(c*c + d*d +f*f) 
     angle = dotProduct/modOfVector1
     print("Cosθ =",angle)
     angleInDegree = math.degrees(math.acos(angle))
     print("θ =",angleInDegree,"°")
angle_of_vectors(v1[0],v1[1],v2[0],v2[1])  
"""   

In [ ]:
#p2 destination , p1 source
def slope(p1,p2):
    x = p2[0]-p1[0] # 0 ise?
    y = p2[1]-p1[1]
    if x == 0:
        return 0.01
    else:
        return y/x

In [ ]:
def foo(contour):
    corners = []
    for i in range(len(contour)-2): #contour[i] = [[144  49]]
        
        p1 = contour[i][0]  # [144  49]
        p2 = contour[i+1][0]
        p3 = contour[i+2][0]
        v1 = vector_of_points(p1,p2)
        v2 = vector_of_points(p1,p3)
        #ang = angle_between(v1,v2)
        #print("i is" + str(i))
        #angle_of_vectors(v1[0],v1[1],v2[0],v2[1])   
        s1 = slope(p1,p2) 
        #print("S1 İS : " + str(s1))
        s2 = slope(p1,p3) 
        #print("S2 İS : " + str(s2))
        ang = abs(s1-s2)
        #print("i is : "+ str(i) + " " + str(ang))
        if ang > 30 or s1*s2<0:
            corners.append(p2)
        #np_corners = np.array(corners)
        #print(corners)
    return corners

In [ ]:
#lex sort kullan
def sort_lex(c_arr):
    
    size = len(c_arr)
    first_column = np.zeros((size), dtype='float64')
    second_column = np.zeros((size), dtype='float64')
    
    for i in range(len(c_arr)): #contour[i] = [[144  49]]
        # contour[i][0] == [144  49]
        first_el = c_arr[i][0][0] 
        second_el = c_arr[i][0][1] 
        first_column[i] = first_el
        second_column[i] = second_el
    
    # Sort by a then by b
    ind = np.lexsort((second_column, first_column))
    #Sorted indices-> [ 7  8  5  6  3  4  9  2 10 11  1  0 24 23 22 12 21 20 13 19 18 15 14 17 16]
    
    ret_arr = np.empty((size,2)) # [[],[]] 2d not 3d
    for i in range(len(ind)):
        sub = [first_column[ind[i]],second_column[ind[i]]]
        ret_arr[i] = sub
        
    return ret_arr    
    

    

In [ ]:
#min ve max bulup yarısını al yada median al
def create_ssd_list(contour):
    size = len(contour)
    ssd_list = np.zeros((size), dtype='float64')
    #ssd_list[size]
    for i in range(len(contour)-1): #contour[i] = [[144  49]]
        first = contour[i][0] # [144  49]
        second = contour[i+1][0] 
        num = ssd(first,second)  
        ssd_list[i] = num
    return ssd_list

def mean_difference_ssd(ssd_list):
    return statistics.mean(ssd_list)
    
def median_difference_ssd(ssd_list):
    return statistics.median(ssd_list)

In [ ]:
"""
Shi-Tomasi method
# import the required library
import numpy as np
import cv2
from matplotlib import pyplot as plt
  
  
# read the image
img = cv2.imread('corner1.png')
  
# convert image to gray scale image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
# detect corners with the goodFeaturesToTrack function.
corners = cv2.goodFeaturesToTrack(gray, 27, 0.01, 10)
corners = np.int0(corners)
  
# we iterate through each corner, 
# making a circle at each point that we think is a corner.
for i in corners:
    x, y = i.ravel()
    cv2.circle(img, (x, y), 3, 255, -1)
  
plt.imshow(img), plt.show()"""

In [ ]:
"""
# canny edge kullanmak istersem

t_lower = 50  # Lower Threshold
t_upper = 150  # Upper threshold
  
# Applying the Canny Edge filter
edge = cv2.Canny(img_normal_blurred, t_lower, t_upper)
cv2.imshow('edge', edge)
cv2.waitKey(0)
cv2.destroyAllWindows()

contours, hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# len contours == number of objects
# canny edge ile 7 tane obje buldu(4 yerine)
"""

In [ ]:
def douglas_peucker(points, epsilon):
    if len(points) <= 2:
        return points

    dmax = 0
    index = 0

    for i in range(1, len(points) - 1):
        d = perpendicular_distance(points[i], points[0], points[-1])
        if d > dmax:
            index = i
            dmax = d

    if dmax > epsilon:
        part1 = douglas_peucker(points[:index + 1], epsilon)
        part2 = douglas_peucker(points[index:], epsilon)

        return np.vstack((part1[:-1], part2))
    else:
        return np.array([points[0], points[-1]])

def perpendicular_distance(point, line_start, line_end):
    if np.all(line_start == line_end):
        return np.linalg.norm(point - line_start)

    return np.abs(np.linalg.norm(np.cross(line_end - line_start, line_start - point))) / np.linalg.norm(line_end - line_start)

def get_polygon_type(points):
    num_vertices = len(points)
    if num_vertices == 3:
        return "Triangle"
    elif num_vertices == 4:
        return "Rectangular"
    elif num_vertices == 5:
        return "Pentagon"
    elif num_vertices == 6:
        return "Hexagon"
    else:
        return "Polygon or circle" #circle da daha az olması gerekir gibi?
                #circle contour çizimini güzelleştir
